# Homework 2: Convolutional Neural Networks for Text Classification

In this homework, you will be implementing the _forward pass_ and _backpropagation_ for a convolutional neural network with sparse inputs for text classification. 

## The setup
Let's define parameters for the Convolutional Neural Network. You do not need to modify them.

In [0]:
import sys
import math
import numpy as np
from collections import defaultdict
import time

# window size for the CNN
width = 2

# number of filters
F = 100

# learning rate
alpha = 1e-1

# vocabsize: size of the total vocabulary
vocabsize = 10000

# vocab: the vocabulary dictionary with the word as key and its index as value
# the input will be transformed into respective positional indices using the vocab dictionary
# as the input for the forward and backward algorithm
# e.g. if vocab = {'a': 0, 'simple': 1, 'sentence': 2} and the training data is
# "a simple simple sentence a",
# the input to the forward and backward algorithm will be [0,1,1,2,0]
vocab = {}

np.random.seed(1)

# U and V are weight vectors of the hidden layer
# U: a matrix of weights of all inputs for the first
# hidden layer for all F filters in the
# where each filter has the size of vocabsize by width (window size)
# U[i, j, k] represents the weight of filter u_j
# for word with vocab[word] = i when the word is
# at the position k of the sliding window
# e.g. for the example, "a simple simple sentence a",
# if the window size is 4 and we are looking at the first sliding window
# of the 9th filter, the weight for the last "sentence" will be U[2, 8, 3]
# i.e U[index of the word in vocab, index of the filter, position of the word in that sliding window]
U = np.random.normal(loc=0, scale=0.01, size=(vocabsize, F, width))
# V: the the weight vector of the F filter outputs (after max pooling)
# that will produce the output, i.e. o = sigmoid(V*h)
V = np.random.normal(loc=0, scale=0.01, size=(F))

Let's define some utility functions that may be useful. You don't need to modify them.

In [0]:
def sigmoid(x):
    """
    helper function that computes the sigmoid function
    """
    return 1. / (1 + math.exp(-x))


def read_vocab(filename):
    """
    helper function that builds up the vocab dictionary for input transformation
    """
    file = open(filename)
    for line in file:
        cols = line.rstrip().split("\t")
        word = cols[0]
        idd = int(cols[1])
        vocab[word] = idd
    file.close()


def read_data(filename):
    """
    :param filename: the name of the file
    :return: list of tuple ([word index list], label)
    as input for the forward and backward function
    """
    data = []
    file = open(filename)
    for line in file:
        cols = line.rstrip().split("\t")
        label = int(cols[0])
        words = cols[1].split(" ")
        w_int = []
        for w in words:
            # skip the unknown words
            if w in vocab:
                w_int.append(vocab[w])
        data.append((w_int, label))
    file.close()
    return data


def train():
    """
    main caller function that reads in the names of the files
    and train the CNN to classify movie reviews
    """
    vocabFile = "vocab.txt"
    trainingFile = "movie_reviews.train"
    testFile = "movie_reviews.dev"

    read_vocab(vocabFile)
    training_data = read_data(trainingFile)
    test_data = read_data(testFile)

    for i in range(50):
        # confusion matrix showing the accuracy of the algorithm
        confusion_training = np.zeros((2, 2))
        confusion_validation = np.zeros((2, 2))

        for (data, label) in training_data:
            # back propagation to update weights for both U and V
            backward(data, label)

            # calculate forward and evaluate
            prob = forward(data)["prob"]
            pred = 1 if prob > .5 else 0
            confusion_training[pred, label] += 1

        for (data, label) in test_data:
            # calculate forward and evaluate
            prob = forward(data)["prob"]
            pred = 1 if prob > .5 else 0
            confusion_validation[pred, label] += 1

        print("Epoch: {} \tDev accuracy: {:.3f}"
            .format(
            i,
            np.sum(np.diag(confusion_validation)) / np.sum(confusion_validation)))

And finally, we'll download the data. We'll be doing sentiment analysis on a dataset of movie reviews, so we'll need 3 files - a vocabulary file, a file with a training set of movie reviews, and a development set containing different reviews.

In [0]:
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/vocab.txt 
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/movie_reviews.dev
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_2/movie_reviews.train

## 1. Forward

Given the parameters and definition of the CNN model (§2 of HW), complete the Forward Function to calculate _o_ (the probability of the positive class) for an input text. You may not import any additional libraries. 

In [0]:
def forward(word_indices):

    """
    :param word_indices: a list of word indices, i.e. idx = vocab[word]
    :return: a result dictionary containing 3 items -
    result['prob']: output of the CNN algorithm.
    result['h']: the hidden layer output after max pooling, h = [h1, ..., hF]
    result['hid']: argmax of F filters, e.g. j of x_j
    e.g. for the ith filter u_i, tanh(word[hid[i], hid[i] + width]*u_i) = max(h_i)
    """
    h = np.zeros(F, dtype=float)
    hid = np.zeros(F, dtype=int)
    prob = 0.0

    # step 1. compute h and hid
    # loop through the input data of word indices and
    # keep track of the max filtered value h_i and its position index x_j
    # h_i = max(tanh(weighted sum of all words in a given window)) over all windows for u_i
    """
    Type your code below
    """
    start_time=time.time()
    global U,V
    #word_indices=np.array([word_indices]).reshape(-1)
    
    conv_output_matrix=np.zeros((F,1))
    #print(len(word_indices))
    for i in range(len(word_indices)-width+1):
      val1= U[word_indices[i],:,0].reshape(-1,1)
      #print("val1:{}".format(val1.shape))
      val2=U[word_indices[i+1],:,1].reshape(-1,1)
      conv_sum=val1+val2
      #print("conv_sum:{}".format(conv_sum.shape))
      if(i==0):
        conv_output_matrix=conv_output_matrix + conv_sum
      else:
        conv_output_matrix=np.concatenate((conv_output_matrix,conv_sum), axis=1)
    #print("final shape:{}".format(conv_output_matrix.shape))
    conv_output_matrix=np.tanh(conv_output_matrix)
    h=np.amax(conv_output_matrix,axis=1) 
    hid=np.argmax(conv_output_matrix,axis=1)
    prob=sigmoid(h@V)
    #print("prob:{0}, h:{1}, hid:{2}".format(prob, h, hid))
    #print("Forward:{}".format(time.time()-start_time))
    return {"prob": prob, "h": h, "hid": hid}

## 2. Backward

Using the gradient update equations for V (§3 in HW) and U (§3.1), implement the updates for U and V in the backward function.

In [0]:
def backward(word_indices, true_label):
    """
    :param word_indices: a list of word indices, i.e. idx = vocab[word]
    :param true_label: true label (0, 1) of the movie reviews
    :return: None
    update weight matrix/vector U and V based on the loss function
    """
    start_time=time.time()
    global U, V
    pred = forward(word_indices)
    prob = pred["prob"]
    h = pred["h"]
    hid = pred["hid"]

    # update U and V here
    # loss_function = y * log(o) + (1 - y) * log(1 - o)
    #               = true_label * log(prob) + (1 - true_label) * log(1 - prob)
    # to update V: V_new =    + d(loss_function)/d(V)*alpha
    # to update U: U_new = U_current + d(loss_function)/d(U)*alpha
    # Make sure you only update the appropriate argmax term for U
    """
    Type your code below
    """
    #one_hot_input = np.eye(vocabsize)[word_indices]
    #print(one_hot_input.shape)
    V_new= V + (true_label-prob)*h*alpha
    dU=np.zeros((vocabsize,F, width))
    for i in range(len(hid)):
      dU[word_indices[hid[i]],i,0]= (float(true_label-prob)) * V[i]*(1-(h[i]**2)) * 1 #h[i] is the tanh(Xmax*U)
      dU[word_indices[hid[i]+1],i,1]= (float(true_label-prob)) * V[i]*(1-(h[i]**2)) * 1 #h[i] is the tanh(Xmax*U) 
    assert dU.shape==U.shape
    V=V_new
    U=U + dU*alpha
    elapsedtime=time.time()-start_time
    #print("Backward time:{}".format(elapsedtime))


Once you have implemented both the forward and backward functions, your can test out your implementations by training the model. To do so, run the `train` function in the cell below. If your implementations are correct, you should see the accuracy improve as the model trains (You will be graded based on the correctness of the implementations, not on this accuracy).

In [16]:
train()

Epoch: 0 	Dev accuracy: 0.620
Epoch: 1 	Dev accuracy: 0.602
Epoch: 2 	Dev accuracy: 0.668
Epoch: 3 	Dev accuracy: 0.698
Epoch: 4 	Dev accuracy: 0.722
Epoch: 5 	Dev accuracy: 0.804
Epoch: 6 	Dev accuracy: 0.806
Epoch: 7 	Dev accuracy: 0.806
Epoch: 8 	Dev accuracy: 0.804
Epoch: 9 	Dev accuracy: 0.806
Epoch: 10 	Dev accuracy: 0.806
Epoch: 11 	Dev accuracy: 0.806
Epoch: 12 	Dev accuracy: 0.810
Epoch: 13 	Dev accuracy: 0.808
Epoch: 14 	Dev accuracy: 0.810
Epoch: 15 	Dev accuracy: 0.810
Epoch: 16 	Dev accuracy: 0.812
Epoch: 17 	Dev accuracy: 0.812
Epoch: 18 	Dev accuracy: 0.814
Epoch: 19 	Dev accuracy: 0.814
Epoch: 20 	Dev accuracy: 0.816
Epoch: 21 	Dev accuracy: 0.816
Epoch: 22 	Dev accuracy: 0.816
Epoch: 23 	Dev accuracy: 0.816
Epoch: 24 	Dev accuracy: 0.818
Epoch: 25 	Dev accuracy: 0.818
Epoch: 26 	Dev accuracy: 0.818
Epoch: 27 	Dev accuracy: 0.818
Epoch: 28 	Dev accuracy: 0.818
Epoch: 29 	Dev accuracy: 0.818
Epoch: 30 	Dev accuracy: 0.818
Epoch: 31 	Dev accuracy: 0.818
Epoch: 32 	Dev acc